<a href="https://colab.research.google.com/github/yf591/Using-stable-diffusion-webui-in-Google-Colab/blob/main/stable_diffusion_webui_AUTOMATIC1111_v1_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Download necessary models and launch stable-diffusion-webui AUTOMATIC1111
- All downloads are handled individually

## 1.Preparation

In [ ]:
# Mount Google Drive (if necessary)
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!git clone https://github.com/AUTOMATIC1111/stable-diffusion-webui
%cd stable-diffusion-webui

In [ ]:
from google.colab import output

# Initial installation of required libraries may be time-consuming due to downloading dependencies.
!pip install -r requirements_versions.txt
!pip install -U xformers
!pip install torch==2.5.0+cu124 torchvision==0.16.0+cu124 torchaudio==2.5.0+cu124 torchtext==0.16.0 torchdata==0.6.0 --extra-index-url https://download.pytorch.org/whl/cu124
!pip install wandb==0.15.12
!pip install -U requests

output.clear()

In [ ]:
import os
import time

In [ ]:
#@title ### Setting API Keys (Hugging Face, Civitai)

# from getpass import getpass
# # Paste your Hugging Face token here (input will be hidden)
# HF_TOKEN = getpass("Enter your Hugging Face read token: ")

from google.colab import userdata

# Get Hugging Face Access Token (HF_TOKEN) if it exists
api_key_hugging_face = userdata.get('HF_TOKEN')
if api_key_hugging_face is None:
    print("Error: Hugging Face Access Tokens secret is not set.")

# Get CIVITAI_TOKEN if it exists
api_key_civitai = userdata.get('CIVITAI_TOKEN')
if api_key_civitai is None:
    print("Error: CIVITAI_API_KEY secret is not set.")

## 2.Select Base Models

#### From Hugging Face

In [ ]:
#@title ### **For Base Model（From Hugging Face）**

dest_dir = "models/Stable-diffusion/"

def download_model(use_model, model_url, model_name):
    if use_model:
        dest_path = os.path.join(dest_dir, model_name)
        print(f"Downloading {model_name}...")
        !wget {model_url} -O {dest_path}
        print(f"{model_name} downloaded.")

start_time = time.time()


#@markdown ## **SD1.5系モデルのダウンロード**

#@markdown ### ◆ sd-v1-5-pruned-noema-fp16.safetensors
use_sd15_pruned = False #@param {type: "boolean"}
download_model(use_sd15_pruned, "https://huggingface.co/hollowstrawberry/stable-diffusion-guide/resolve/main/models/sd-v1-5-pruned-noema-fp16.safetensors", "sd-v1-5-pruned-noema-fp16.safetensors")

#@markdown ### ◆ chilled_remix_v2.safetensors
use_chilled_remix = False #@param {type: "boolean"}
download_model(use_chilled_remix, "https://huggingface.co/sazyou-roukaku/chilled_remix/resolve/main/chilled_remix_v2.safetensors", "chilled_remix_v2.safetensors")

#@markdown ### ◆ majicmixRealistic_v6.safetensors
use_majicmix_realistic = False #@param {type: "boolean"}
download_model(use_majicmix_realistic, "https://huggingface.co/casque/majicmixRealistic_v6/resolve/main/majicmixRealistic_v6.safetensors", "majicmixRealistic_v6.safetensors")

#@markdown ---

#@markdown ## **SDXL系モデルのダウンロード**

#@markdown ### ◆ stabilityai/stable-diffusion-xl-base-1.0
use_sdxl_base = False #@param {type: "boolean"}
download_model(use_sdxl_base, "https://huggingface.co/stabilityai/stable-diffusion-xl-base-1.0/resolve/main/sd_xl_base_1.0.safetensors", "sd_xl_base_1.0.safetensors")

#@markdown ### ◆ stabilityai/stable-diffusion-xl-refiner-1.0 ※画質向上に役立つ
use_sdxl_refiner = False #@param {type: "boolean"}
download_model(use_sdxl_refiner, "https://huggingface.co/stabilityai/stable-diffusion-xl-refiner-1.0/resolve/main/sd_xl_refiner_1.0.safetensors", "sd_xl_refiner_1.0.safetensors")

'''
#@markdown ---
#@markdown ##※Refinerの設定について

#@markdown ①「txt2img」タブから、「Refiner」をクリック
#@markdown  - Stable diffusion web uiのRefiner選択画面

#@markdown ②「Checkpoint」の更新アイコンをクリックし、プルダウンメニューからダウンロードしたRefinerを選択. Switch at: 0.8

#@markdown ---
'''


end_time = time.time()
elapsed_time = end_time - start_time
print(f"すべての処理が完全に完了しました！({elapsed_time:.2f}秒)")

#### From Civitai

In [ ]:
#@title ### **For Base Model（From CIvitai）**


#@markdown ## **Civitai Model Download**

dest_dir = "/content/stable-diffusion-webui/models/Stable-diffusion"

def download_civitai_model(use_model, model_url, model_name, api_key_civitai=None):
    if use_model:
        model_file_path = os.path.join(dest_dir, model_name)
        if not os.path.exists(dest_dir):
            os.makedirs(dest_dir)
            print(f"Directory '{dest_dir}' created.")
        if not os.path.exists(model_file_path):
            print(f"Downloading {model_name}...")
            if api_key_civitai:
                !wget -O {model_file_path} {model_url.format(token=api_key_civitai)}
            else:
                !wget -O {model_file_path} {model_url.format(token='')}
            print(f"{model_name} downloaded.")
        else:
            print(f"{model_name} already downloaded.")

start_time = time.time()

"""
#@markdown ---

#@markdown ※シークレットにCivitaiのAPIキーを保存しておいてください。

#@markdown ---
"""
#@markdown ## **実写系モデル(Base: SD 1.5) / From Civitai**

#@markdown ### [Noble Mix Fix](https://civitai.com/models/65480/noble-mix-fix)
#@markdown ##### 　※License:[CreativeML Open RAIL-M](https://huggingface.co/spaces/CompVis/stable-diffusion-license) [Addendum](https://civitai.com/models/license/279964) ※クレジット表記不要・商用可・マージを共有するときに異なる権限可
use_noble_mix_fix = True #@param {type: "boolean"}
model_url = "https://civitai.com/api/download/models/92828?token={token}"
download_civitai_model(use_noble_mix_fix, model_url, "noble_mix_fix.safetensors",api_key_civitai)


#@markdown ### [RealBeautyMix](https://civitai.com/models/85156)
#@markdown ##### 　※License:[CreativeML Open RAIL-M](https://huggingface.co/spaces/CompVis/stable-diffusion-license) [Addendum](https://civitai.com/models/license/90505) ※クレジット表記不要・商用可・マージを共有するときに異なる権限可
use_real_beauty_mix = False #@param {type: "boolean"}
model_url = "https://civitai.com/api/download/models/90505?token={token}"
download_civitai_model(use_real_beauty_mix, model_url, "RealBeautyMix.safetensors",api_key_civitai)


#@markdown ---

#@markdown ## **イラスト系モデル (Base: SD 1.5)** / From Civitai

#@markdown ### [MND-Mix](https://civitai.com/models/27259/tmnd-mix)
#@markdown ##### 　※License:[CreativeML Open RAIL-M](https://huggingface.co/spaces/CompVis/stable-diffusion-license) [Addendum](https://civitai.com/models/license/221220) ※クレジット表記不要・商用不可・マージを共有するときに異なる権限可
use_TMND_Mix = False #@param {type:"boolean"}
model_url = "https://civitai.com/api/download/models/221220?type=Model&format=SafeTensor&size=pruned&fp=fp16?token={token}"
download_civitai_model(use_TMND_Mix, model_url, "TMND-Mix.safetensors", api_key_civitai)


#@markdown ---

#@markdown ## **イラスト系モデル (Base: SDXL)** / From Civitai

#@markdown ### [AAM_XL_AnimeMix](https://civitai.com/models/269232?modelVersionId=303526)
#@markdown ##### 　※License:[CreativeML Open RAIL++-M](https://github.com/Stability-AI/generative-models/blob/main/model_licenses/LICENSE-SDXL1.0) [Addendum](https://civitai.com/models/license/303526) ※クレジット表記不要・商用可・マージを共有するときに異なる権限可
use_AAM_XL_AnimeMix = False #@param {type:"boolean"}
model_url = "https://civitai.com/api/download/models/303526?type=Model&format=SafeTensor&size=full&fp=fp16?token={token}"
download_civitai_model(use_AAM_XL_AnimeMix, model_url, "AAM_XL_AnimeMix.safetensors", api_key_civitai)

#@markdown ---





end_time = time.time()
elapsed_time = end_time - start_time
print(f"すべての処理が完全に完了しました！({elapsed_time:.2f}秒)")

#### From MyDrive

In [ ]:
#@title ### **For Base Model（From MyDrive）**

#@markdown ## **From MyDrive（ローカルにDLしたBaseModel） / Base: SD 1.5**

base_dir = "/content/drive/MyDrive/Colab_Notebooks/Stable_Dffusion/stable-diffusion-webui/models/Stable-diffusion/"
dest_dir = "models/Stable-diffusion/"

def copy_model(use_model, model_name):
  if use_model:
    source_path = os.path.join(base_dir, model_name)
    dest_path = os.path.join(dest_dir, model_name)
    print(f"Copying {model_name}...")
    !cp {source_path} {dest_path}
    print(f"{model_name} copied.")

start_time = time.time()

#@markdown ### 1. [grapefruitHentaiModel_grapefruitv41](https://civitai.com/models/24383/grapefruit-hentai-model)
#@markdown ##### 　※License:[CreativeML Open RAIL-M](https://huggingface.co/spaces/CompVis/stable-diffusion-license) [Addendum](https://civitai.com/models/license/29179) ※クレジット表記不要・商用可・マージを共有するときに異なる権限可
use_grapefruitHentaiModel_grapefruitv41 = False #@param {type: "boolean"}
copy_model(use_grapefruitHentaiModel_grapefruitv41, "grapefruitHentaiModel_grapefruitv41.safetensors")

#@markdown ### 2. [hentaimodel_v10](https://civitai.com/models/309915/hentaimodel)
#@markdown ##### 　※License:[CreativeML Open RAIL-M](https://huggingface.co/spaces/CompVis/stable-diffusion-license) [Addendum](https://civitai.com/models/license/347846) ※クレジット表記不要・商用可・マージを共有するときに異なる権限可
use_hentaimodel_v10 = False #@param {type: "boolean"}
copy_model(use_hentaimodel_v10, "hentaimodel_v10.safetensors")

#@markdown ### 4. [anyloraCheckpoint_bakedvaeBlessedFp16](https://civitai.com/models/23900/anylora-checkpoint)
#@markdown ##### 　※License:[CreativeML Open RAIL-M](https://huggingface.co/spaces/CompVis/stable-diffusion-license) [Addendum](https://civitai.com/models/license/95489) ※クレジット表記不要・商用可・マージを共有するときに異なる権限可
use_anyloraCheckpoint_bakedvaeBlessedFp16 = False #@param {type: "boolean"}
copy_model(use_anyloraCheckpoint_bakedvaeBlessedFp16, "anyloraCheckpoint_bakedvaeBlessedFp16.safetensors")

#@markdown ### 6. [pikasNewGeneration_v20](https://civitai.com/models/47067)
#@markdown ##### 　※License:[CreativeML Open RAIL-M](https://huggingface.co/spaces/CompVis/stable-diffusion-license) [Addendum](https://civitai.com/models/license/71733) ※クレジット表記不要・商用可・マージを共有するときに異なる権限可
use_pikasNewGeneration_v20 = False #@param {type: "boolean"}
copy_model(use_pikasNewGeneration_v20, "pikasNewGeneration_v20.safetensors")


#@markdown ---

#@markdown ## **From MyDrive（自作したBaseModel） / Base: SD 1.5**

#@markdown ### 1. merged_model_chillre_majic
use_merged_model_chillre_majic = True #@param {type: "boolean"}
copy_model(use_merged_model_chillre_majic, "merged_model_chillre_majic.safetensors")

#@markdown ### 3. merged_model_CMRbm
use_merged_model_CMRbm = False #@param {type: "boolean"}
copy_model(use_merged_model_CMRbm, "merged_model_CMRbm.safetensors")

#@markdown ### 5. merged_model_2samples_M60_R40
use_merged_model_2samples_M60_R40 = False #@param {type: "boolean"}
copy_model(use_merged_model_2samples_M60_R40, "merged_model_2samples_M60_R40.safetensors")

#@markdown ### 7. merged_model_4samples
use_merged_model_4samples = False #@param {type: "boolean"}
copy_model(use_merged_model_4samples, "merged_model_4samples.safetensors")

#@markdown ### 8. merged_model_3samples_N40_R30_M30
use_merged_model_3samples_N40_R30_M30 = True #@param {type: "boolean"}
copy_model(use_merged_model_3samples_N40_R30_M30, "merged_model_3samples_N40_R30_M30.safetensors")


end_time = time.time()
elapsed_time = end_time - start_time
print(f"すべての処理が完全に完了しました！({elapsed_time:.2f}秒)")

## 3.Select VAE

In [ ]:
#@title ### **VAE Download**

#@markdown ## **VAE Download**

vae_dir = "/content/stable-diffusion-webui/models/VAE/"

def download_vae(use_vae, vae_name, vae_url):
    if use_vae:
        vae_path = os.path.join(vae_dir, vae_name)
        if not os.path.exists(vae_dir):
            os.makedirs(vae_dir)
            print(f"Directory '{vae_dir}' created.")

        if not os.path.exists(vae_path):
            print(f"Downloading {vae_name}...")
            !wget -O {vae_path} {vae_url}
            print(f"{vae_name} downloaded.")
        else:
            print(f"{vae_name} already downloaded.")

start_time = time.time()

#@markdown ### vae-ft-mse-840000-ema-pruned.safetensors
use_vae_ft_mse = True #@param {type:"boolean"}
download_vae(use_vae_ft_mse, "vae-ft-mse-840000-ema-pruned.safetensors", "https://huggingface.co/stabilityai/sd-vae-ft-mse-original/resolve/main/vae-ft-mse-840000-ema-pruned.safetensors")

#@markdown ### sdxl_vae.safetensors
use_sdxl_vae = False #@param {type:"boolean"}
download_vae(use_sdxl_vae, "sdxl_vae.safetensors", "https://huggingface.co/stabilityai/sdxl-vae/resolve/main/sdxl_vae.safetensors")

end_time = time.time()
elapsed_time = end_time - start_time
print(f"すべての処理が完全に完了しました！({elapsed_time:.2f}秒)")

## 4.Select Lora Models

In [ ]:
#@title ### **For LoRA Model（From MyDrive）**

# LoRAファイルを配置するディレクトリ
lora_dir = "models/Lora"

# ディレクトリが存在するか確認し、なければ作成
if not os.path.exists(lora_dir):
    os.makedirs(lora_dir)
    print(f"Directory '{lora_dir}' created.")
elif not os.path.isdir(lora_dir):
    print(f"Error: '{lora_dir}' is not a directory. Please remove the existing file and try again.")
else:
    print(f"Directory '{lora_dir}' already exists.")


base_dir = "/content/drive/MyDrive/sd-webui-google-colab-setup/stable-diffusion-webui/models/loras/"
dest_dir = "models/Lora/"

def copy_lora_model(use_model, model_name, sub_dir=""):
    if use_model:
        source_path = os.path.join(base_dir, sub_dir, model_name)
        dest_path = os.path.join(dest_dir, model_name)
        print(f"Copying {model_name}...")
        !cp {source_path} {dest_path}
        print(f"{model_name} copied.")


start_time = time.time()

#@markdown ## **From MyDrive（ローカルにDLしたLoRAモデル）**

#@markdown ### 5. [PovGroupSex_v10](https://civitai.com/models/114843/pov-group-sex-or-sex-with-multiple-girls)
#@markdown ##### 　※License: ※クレジット表記不要・商用可・マージを共有するときに異なる権限可
use_PovGroupSex_v10 = True #@param {type:"boolean"}
copy_lora_model(use_PovGroupSex_v10,"PovGroupSex_v10.safetensors")

#@markdown ### 6. [MultipleGirlsGroup](https://civitai.com/models/51136/multiple-girls-group)
#@markdown ##### 　※License: ※クレジット表記不要・商用可・マージを共有するときに異なる権限可
use_MultipleGirlsGroup = True #@param {type:"boolean"}
copy_lora_model(use_MultipleGirlsGroup,"MultipleGirlsGroup.safetensors")

#@markdown ### 7. [style21](https://civitai.com/models/135903/anime-style-box)
#@markdown ##### 　※License: ※クレジット表記不要・商用可・マージを共有するときに異なる権限可
use_style21 = True #@param {type:"boolean"}
copy_lora_model(use_style21,"style21.safetensors")

#@markdown ### 8. [animemix_v3_offset](https://civitai.com/models/4982/anime-screencap-style-lora)
#@markdown ##### 　※License: ※クレジット表記不要・商用可・マージを共有するときに異なる権限可
use_animemix_v3_offset = True #@param {type:"boolean"}
copy_lora_model(use_animemix_v3_offset,"animemix_v3_offset.safetensors")

#@markdown ### 9. [eropose_allfours](https://civitai.com/models/95579/eropose-all-fours)
#@markdown ##### 　※License: ※クレジット表記不要・商用可・マージを共有するときに異なる権限不可
use_eropose_allfours = True #@param {type:"boolean"}
copy_lora_model(use_eropose_allfours,"eropose_allfours.safetensors")

#@markdown ### 15. [Full Body/All fours Fellatio | LoRA](https://civitai.com/models/80122/full-bodyall-fours-fellatio-or-lora)
#@markdown ##### 　※License: ※クレジット表記不要・商用可・マージを共有するときに異なる権限可
use_Full_Body_All_fours_Fellatio = True #@param {type:"boolean"}
copy_lora_model(use_Full_Body_All_fours_Fellatio,"BJ_1.safetensors")

#@markdown ---

#@markdown ## **From MyDrive（自作したLoRAモデル）**

#@markdown ### ◆ 2. AmazonPhotos_Sn

#@markdown #### - AmazonPhotos_Sn-15
use_AmazonPhotos_Sn_15 = True #@param {type:"boolean"}
copy_lora_model(use_AmazonPhotos_Sn_15,"AmazonPhotos_Sn-15.safetensors", sub_dir="AmazonPhotos_Sn/output")

#@markdown #### - AmazonPhotos_Sn-20
use_AmazonPhotos_Sn_20 = True #@param {type:"boolean"}
copy_lora_model(use_AmazonPhotos_Sn_20,"AmazonPhotos_Sn-20.safetensors", sub_dir="AmazonPhotos_Sn/output")

#@markdown #### - AmazonPhotos_Sn-25
use_AmazonPhotos_Sn_25 = True #@param {type:"boolean"}
copy_lora_model(use_AmazonPhotos_Sn_25,"AmazonPhotos_Sn-25.safetensors", sub_dir="AmazonPhotos_Sn/output")


#@markdown ### ◆ 4. AmazonPhotos_2_Sn

#@markdown #### - AmazonPhotos_2_Sn-15
use_AmazonPhotos_2_Sn_15 = True #@param {type:"boolean"}
copy_lora_model(use_AmazonPhotos_2_Sn_15,"AmazonPhotos_2_Sn-15.safetensors", sub_dir="AmazonPhotos_2_Sn/output")

#@markdown #### - AmazonPhotos_2_Sn-20
use_AmazonPhotos_2_Sn_20 = True #@param {type:"boolean"}
copy_lora_model(use_AmazonPhotos_2_Sn_20,"AmazonPhotos_2_Sn-20.safetensors", sub_dir="AmazonPhotos_2_Sn/output")

#@markdown #### - AmazonPhotos_2_Sn-25
use_AmazonPhotos_2_Sn_25 = True #@param {type:"boolean"}
copy_lora_model(use_AmazonPhotos_2_Sn_25,"AmazonPhotos_2_Sn-25.safetensors", sub_dir="AmazonPhotos_2_Sn/output")


#@markdown ### ◆ 10. CuteGirl_Mix_Sn_Lora

#@markdown #### - CuteGirl_Mix_Sn_Lora-05
use_CuteGirl_Mix_Sn_Lora_05 = True #@param {type:"boolean"}
copy_lora_model(use_CuteGirl_Mix_Sn_Lora_05,"CuteGirl_Mix_Sn_Lora-05.safetensors", sub_dir="CuteGirl_Mix_Sn_Lora/output")

#@markdown #### - CuteGirl_Mix_Sn_Lora-10
use_CuteGirl_Mix_Sn_Lora_10 = True #@param {type:"boolean"}
copy_lora_model(use_CuteGirl_Mix_Sn_Lora_10,"CuteGirl_Mix_Sn_Lora-10.safetensors", sub_dir="CuteGirl_Mix_Sn_Lora/output")

#@markdown #### - CuteGirl_Mix_Sn_Lora-15
use_CuteGirl_Mix_Sn_Lora_15 = True #@param {type:"boolean"}
copy_lora_model(use_CuteGirl_Mix_Sn_Lora_15,"CuteGirl_Mix_Sn_Lora-15.safetensors", sub_dir="CuteGirl_Mix_Sn_Lora/output")

#@markdown ### ◆ 11. CuteGirl_Mix2_Sn_Lora

#@markdown #### - CuteGirl_Mix2_Sn_Lora-10
use_CuteGirl_Mix2_Sn_Lora_10 = True #@param {type:"boolean"}
copy_lora_model(use_CuteGirl_Mix2_Sn_Lora_10,"CuteGirl_Mix2_Sn_Lora-10.safetensors", sub_dir="CuteGirl_Mix2_Sn_Lora/output")

#@markdown #### - CuteGirl_Mix2_Sn_Lora-15
use_CuteGirl_Mix2_Sn_Lora_15 = True #@param {type:"boolean"}
copy_lora_model(use_CuteGirl_Mix2_Sn_Lora_15,"CuteGirl_Mix2_Sn_Lora-15.safetensors", sub_dir="CuteGirl_Mix2_Sn_Lora/output")

#@markdown #### - CuteGirl_Mix2_Sn_Lora-20

use_CuteGirl_Mix2_Sn_Lora_20 = True #@param {type:"boolean"}
copy_lora_model(use_CuteGirl_Mix2_Sn_Lora_20,"CuteGirl_Mix2_Sn_Lora-20.safetensors", sub_dir="CuteGirl_Mix2_Sn_Lora/output")


#@markdown ### ◆ 12. CuteGirlMix1-2Lora_Merged_lora
use_CuteGirlMix1_2Lora_Merged_lora = True #@param {type:"boolean"}
copy_lora_model(use_CuteGirlMix1_2Lora_Merged_lora,"CuteGirlMix1-2Lora_Merged_lora.safetensors")


#@markdown ### ◆ 13. all_fours

#@markdown #### - all_fours-10
use_all_fours_10 = True #@param {type:"boolean"}
copy_lora_model(use_all_fours_10,"all_fours-10.safetensors", sub_dir="all_fours/output")

#@markdown #### - all_fours-15
use_all_fours_15 = True #@param {type:"boolean"}
copy_lora_model(use_all_fours_15,"all_fours-15.safetensors", sub_dir="all_fours/output")

#@markdown #### - all_fours-20
use_all_fours_20 = True #@param {type:"boolean"}
copy_lora_model(use_all_fours_20,"all_fours-20.safetensors", sub_dir="all_fours/output")

#@markdown ### ◆ 14. merged_lora_4samples
use_merged_lora_4samples = True #@param {type:"boolean"}
copy_lora_model(use_merged_lora_4samples,"merged_lora_4samples.safetensors")

end_time = time.time()
elapsed_time = end_time - start_time
print(f"すべての処理が完全に完了しました！({elapsed_time:.2f}秒)")

## 5.Select Embedings

In [ ]:
#@title ### **Embedding Download**

import urllib.request

#@markdown ## **Embedding Download**

dest_dir = "/content/stable-diffusion-webui/embeddings"

def download_embedding(use_embedding, embedding_name, embedding_url, use_civitai_api=False, api_key_civitai=""):
    if use_embedding:
        embedding_path = os.path.join(dest_dir, embedding_name)
        if not os.path.exists(dest_dir):
            os.makedirs(dest_dir)
            print(f"Directory '{dest_dir}' created.")

        if not os.path.exists(embedding_path):
            print(f"Downloading {embedding_name}...")
            if use_civitai_api:
              if not api_key_civitai:
                raise ValueError("Civitai API key is not set. Please set CIVITAI_TOKEN in Google Colab secrets.")
              !wget {embedding_url.format(token=api_key_civitai)} --content-disposition --directory-prefix={dest_dir}/
            else:
                urllib.request.urlretrieve(embedding_url, embedding_path)
            print(f"{embedding_name} downloaded.")
        else:
            print(f"{embedding_name} already downloaded.")


start_time = time.time()

"""
#@markdown ---

#@markdown ※シークレットにCivitaiのAPIキーを保存しておいてください。

#@markdown ---
"""

#@markdown ### ◆ [EasyNegative.safetensors](https://huggingface.co/datasets/gsdf/EasyNegative/blob/main/EasyNegative.safetensors)
use_EasyNegative = True #@param {type:"boolean"}
download_embedding(use_EasyNegative, "EasyNegative.safetensors", "https://huggingface.co/datasets/gsdf/EasyNegative/resolve/main/EasyNegative.safetensors")

#@markdown ### ◆ [negative_hand Negative Embedding](https://civitai.com/models/56519/negativehand-negative-embedding)（From Civitai）
use_negative_hand = True #@param {type:"boolean"}
download_embedding(use_negative_hand, "negative_hand-neg.safetensors", "https://civitai.com/api/download/models/60938?token={token}", use_civitai_api=True, api_key_civitai=api_key_civitai)

#@markdown ### ◆ [badhandv4](https://civitai.com/models/16993/badhandv4-animeillustdiffusion)（From Civitai）
use_badhandv4 = True #@param {type:"boolean"}
download_embedding(use_badhandv4, "badhandv4.safetensors", "https://civitai.com/api/download/models/20068?token={token}", use_civitai_api=True, api_key_civitai=api_key_civitai)

end_time = time.time()
elapsed_time = end_time - start_time
print(f"すべての処理が完全に完了しました！({elapsed_time:.2f}秒)")

**Stable Diffusion Web UI での Embedding の使い方**

*   **配置:**
    *   ダウンロードした Embedding ファイル (`.pt` または `.safetensors`) を `stable-diffusion-webui/embeddings` ディレクトリに配置します。
*   **プロンプト:**
    *   プロンプトの中で、使用したい Embedding のファイル名を `<>` で囲んで記述します。
        *   例: `<EasyNegative>`
*   **生成:**
    *   プロンプトに Embedding を記述した状態で画像を生成します。
*   **効果:**
    *   ネガティブプロンプトの強化 (低品質な画像の抑制など)。
    *   特定のスタイルの適用。
    *   プロンプトの簡略化。
*   **注意:**
    *   Embedding ファイル名とプロンプトの記述は正確に一致させる必要があります。
    *   複数の Embedding を同時に使用可能です。
    *   Embedding に適用強度を指定することはできません。

**補足:**

*   Embedding は、LoRA や VAE と異なり、`モデルの選択` のような項目で明示的に選択するものではありません。
*   プロンプトに記述することで、モデルの学習内容に影響を与えます。


## 6.Select Upscaler

In [ ]:
#@title ### **Upscaler**

#@markdown ## **Upscaler**
#@markdown ---

#@markdown ### ◆ [4x-UltraSharp](https://openmodeldb.info/models/4x-UltraSharp)

#@markdown #### - ダウンロードについて

#@markdown こちらはMegaにファイルがホストされている関係でプログラム側から直接ダウンロードが出来ません。[4x-UltraSharp](https://openmodeldb.info/models/4x-UltraSharp)より直接ダウンロードした上で、/content/stable-diffusion-webui/models/ESRGANにファイルを格納してご利用ください。


#@markdown ### ◆ [4x-AnimeSharp](https://openmodeldb.info/models/4x-AnimeSharp)


#@markdown #### - ダウンロードについて

#@markdown こちらはMegaにファイルがホストされている関係でプログラム側から直接ダウンロードが出来ません。[4x-AnimeSharp](https://openmodeldb.info/models/4x-AnimeSharp)より直接ダウンロードした上で、/content/stable-diffusion-webui/models/ESRGANにファイルを格納してご利用ください。

## 7.Select ControlNet

In [ ]:
#@title ### **ControlNet Download**

#@markdown ## **ControlNet Download**

webui_dir = "/content/stable-diffusion-webui"
extension_dir = f"{webui_dir}/extensions"
controlnet_model_dir = f"/content/stable-diffusion-webui/models/ControlNet"


def download_controlnet(use_controlnet, model_name, model_url, is_lora=False):
    if use_controlnet:
        if is_lora:
          model_path = os.path.join(lora_dir, model_name)
        else:
          model_path = os.path.join(controlnet_model_dir, model_name)
        if not os.path.exists(os.path.dirname(model_path)):
            os.makedirs(os.path.dirname(model_path), exist_ok=True)
            print(f"Directory '{os.path.dirname(model_path)}' created.")

        if not os.path.exists(model_path):
            print(f"Downloading {model_name}...")
            !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {model_url} -d {os.path.dirname(model_path)} -o {os.path.basename(model_path)}
            print(f"{model_name} downloaded.")
        else:
            print(f"{model_name} already downloaded.")

def install_controlnet_extension(use_extension):
    if use_extension:
        if not os.path.exists(f"{extension_dir}/sd-webui-controlnet"):
            print("ControlNet拡張機能をインストールします...")
            !git clone https://github.com/Mikubill/sd-webui-controlnet.git {extension_dir}/sd-webui-controlnet
            # requirements.txt がある場合はインストール
            if os.path.exists(f"{extension_dir}/sd-webui-controlnet/requirements.txt"):
                !pip install -r {extension_dir}/sd-webui-controlnet/requirements.txt
            print("ControlNet拡張機能のインストールが完了しました")
        else:
            print("ControlNet拡張機能は既にインストール済みです")

# `aria2c` のインストール
!apt install -y aria2
!pip install protobuf==3.20.2
# IP-Adapter-FaceIDを利用する際に必要になる
!pip install insightface==0.7.3

start_time = time.time()

# ControlNet拡張機能のインストール
#@markdown ### **★ ControlNet Extension**
use_controlnet_extension = False #@param {type: "boolean"}
install_controlnet_extension(use_controlnet_extension)

#@markdown ---
#@markdown ---
#@markdown ### **★ SD 1.5 ControlNet Models（一括インストール）**
#@markdown 以下のモデルは、SD 1.5 のベースモデルで使用できます。
use_control_net_sd15 = False #@param {type: "boolean"}
if use_control_net_sd15:
    model_urls = {
        "control_v11e_sd15_ip2p.pth": "https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11e_sd15_ip2p.pth",
        "control_v11e_sd15_shuffle.pth": "https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11e_sd15_shuffle.pth",
        "control_v11f1e_sd15_tile.pth": "https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11f1e_sd15_tile.pth",
        "control_v11f1p_sd15_depth.pth": "https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11f1p_sd15_depth.pth",
        "control_v11p_sd15_canny.pth": "https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_canny.pth",
        "control_v11p_sd15_inpaint.pth": "https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_inpaint.pth",
        "control_v11p_sd15_lineart.pth": "https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_lineart.pth",
        "control_v11p_sd15_mlsd.pth": "https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_mlsd.pth",
        "control_v11p_sd15_normalbae.pth": "https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_normalbae.pth",
        "control_v11p_sd15_openpose.pth": "https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_openpose.pth",
        "control_v11p_sd15_scribble.pth": "https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_scribble.pth",
        "control_v11p_sd15_seg.pth": "https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_seg.pth",
        "control_v11p_sd15_softedge.pth": "https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_softedge.pth",
        "control_v11p_sd15s2_lineart_anime.pth": "https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15s2_lineart_anime.pth",
        "ioclab_sd15_recolor.safetensors": "https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/ioclab_sd15_recolor.safetensors",
        "ip-adapter_sd15.pth": "https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/ip-adapter_sd15.pth",
        "ip-adapter_sd15_plus.pth": "https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/ip-adapter_sd15_plus.pth",
        "control_sd15_inpaint_depth_hand_fp16.safetensors": "https://huggingface.co/hr16/ControlNet-HandRefiner-pruned/resolve/main/control_sd15_inpaint_depth_hand_fp16.safetensors", # コード1でDLするもの
    }
    for filename, url in model_urls.items():
        download_controlnet(use_control_net_sd15, filename, url)
     # IP-Adapter-FaceID
    face_id_model_urls = {
        "ip-adapter-faceid_sd15.bin": "https://huggingface.co/h94/IP-Adapter-FaceID/resolve/main/ip-adapter-faceid_sd15.bin",
        "ip-adapter-faceid-plus_sd15.bin": "https://huggingface.co/h94/IP-Adapter-FaceID/resolve/main/ip-adapter-faceid-plus_sd15.bin",
        "ip-adapter-faceid-plusv2_sd15.bin": "https://huggingface.co/h94/IP-Adapter-FaceID/resolve/main/ip-adapter-faceid-plusv2_sd15.bin",
        "ip-adapter-faceid-portrait_sd15.bin": "https://huggingface.co/h94/IP-Adapter-FaceID/resolve/main/ip-adapter-faceid-portrait_sd15.bin",
        "ip-adapter-faceid_sd15_lora.safetensors": "https://huggingface.co/h94/IP-Adapter-FaceID/resolve/main/ip-adapter-faceid_sd15_lora.safetensors",
        "ip-adapter-faceid-plus_sd15_lora.safetensors": "https://huggingface.co/h94/IP-Adapter-FaceID/resolve/main/ip-adapter-faceid-plus_sd15_lora.safetensors",
        "ip-adapter-faceid-plusv2_sd15_lora.safetensors": "https://huggingface.co/h94/IP-Adapter-FaceID/resolve/main/ip-adapter-faceid-plusv2_sd15_lora.safetensors"
    }

    for filename, url in face_id_model_urls.items():
       download_controlnet(use_control_net_sd15, filename, url, is_lora=filename.endswith(".safetensors"))

#@markdown ---
#@markdown ### **★ SD 1.5 ControlNet Models（個別に選択してインストール）**
#@markdown 以下のモデルは、SD 1.5 のベースモデルで使用できます。

#@markdown #### 1. control_v11e_sd15_ip2p.pth
#@markdown  画像を別の画像へ変換する際に、元の画像の特徴（色、スタイルなど）を保持しながら変換を行うモデルです。
use_control_v11e_sd15_ip2p = False #@param {type: "boolean"}
download_controlnet(use_control_v11e_sd15_ip2p, "control_v11e_sd15_ip2p.pth", "https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11e_sd15_ip2p.pth")

#@markdown #### 2. control_v11e_sd15_shuffle.pth
#@markdown  画像の要素をシャッフルして新しい構造を生成するモデルです。元の画像の内容を維持しつつ、視覚的な構成を大きく変えたい場合に利用します。
use_control_v11e_sd15_shuffle = False #@param {type: "boolean"}
download_controlnet(use_control_v11e_sd15_shuffle, "control_v11e_sd15_shuffle.pth", "https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11e_sd15_shuffle.pth")

#@markdown #### 3. control_v11f1e_sd15_tile.pth
#@markdown  画像をタイル状に分割し、それぞれのタイルを独立して処理することで、大きな画像を生成する際の繰り返しパターンやテクスチャを制御するのに適したモデルです。
use_control_v11f1e_sd15_tile = False #@param {type: "boolean"}
download_controlnet(use_control_v11f1e_sd15_tile, "control_v11f1e_sd15_tile.pth", "https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11f1e_sd15_tile.pth")

#@markdown #### 4. control_v11f1p_sd15_depth.pth
#@markdown  画像から深度マップ（奥行き情報）を生成するモデルで、この深度情報を基に画像の立体感や奥行きをコントロールすることができます。
use_control_v11f1p_sd15_depth = False #@param {type: "boolean"}
download_controlnet(use_control_v11f1p_sd15_depth, "control_v11f1p_sd15_depth.pth", "https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11f1p_sd15_depth.pth")

#@markdown #### 5. control_v11p_sd15_canny.pth
#@markdown  画像のエッジを検出するモデルで、検出されたエッジを基に画像の構造を制御することができます。
use_control_v11p_sd15_canny = False #@param {type: "boolean"}
download_controlnet(use_control_v11p_sd15_canny, "control_v11p_sd15_canny.pth", "https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_canny.pth")

#@markdown #### 6. control_v11p_sd15_inpaint.pth
#@markdown  画像の特定の部分を修復（インペイント）するモデルで、マスクされた領域を周囲の画像と自然に統合します。
use_control_v11p_sd15_inpaint = False #@param {type: "boolean"}
download_controlnet(use_control_v11p_sd15_inpaint, "control_v11p_sd15_inpaint.pth", "https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_inpaint.pth")

#@markdown #### 7. control_v11p_sd15_lineart.pth
#@markdown  画像から線画を抽出するモデルで、抽出された線画を基にイラストや線画タッチの画像を生成するのに役立ちます。
use_control_v11p_sd15_lineart = False #@param {type: "boolean"}
download_controlnet(use_control_v11p_sd15_lineart, "control_v11p_sd15_lineart.pth", "https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_lineart.pth")

#@markdown #### 8. control_v11p_sd15_mlsd.pth
#@markdown  画像から直線や線分を検出するモデルで、検出された線分に基づいて建物の構造や幾何学的なパターンを制御するのに利用できます。
use_control_v11p_sd15_mlsd = False #@param {type: "boolean"}
download_controlnet(use_control_v11p_sd15_mlsd, "control_v11p_sd15_mlsd.pth", "https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_mlsd.pth")

#@markdown #### 9. control_v11p_sd15_normalbae.pth
#@markdown  画像から法線マップを生成するモデルで、生成された法線マップを基に3Dモデルのようなライティングや陰影を制御することができます。
use_control_v11p_sd15_normalbae = False #@param {type: "boolean"}
download_controlnet(use_control_v11p_sd15_normalbae, "control_v11p_sd15_normalbae.pth", "https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_normalbae.pth")

#@markdown #### 10. control_v11p_sd15_openpose.pth
#@markdown  画像から人物のポーズを検出するモデルで、検出されたポーズに基づいて人物の姿勢を制御できます。
use_control_v11p_sd15_openpose = False #@param {type: "boolean"}
download_controlnet(use_control_v11p_sd15_openpose, "control_v11p_sd15_openpose.pth", "https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_openpose.pth")

#@markdown #### 11. control_v11p_sd15_scribble.pth
#@markdown  手書きの落書きやスケッチを基に画像を生成するモデルで、ラフなスケッチから画像を作成するのに利用できます。
use_control_v11p_sd15_scribble = False #@param {type: "boolean"}
download_controlnet(use_control_v11p_sd15_scribble, "control_v11p_sd15_scribble.pth", "https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_scribble.pth")

#@markdown #### 12. control_v11p_sd15_seg.pth
#@markdown  画像内の各オブジェクトをセグメント化（領域分割）するモデルで、セグメント化された領域を基にオブジェクトの配置や内容を制御できます。
use_control_v11p_sd15_seg = False #@param {type: "boolean"}
download_controlnet(use_control_v11p_sd15_seg, "control_v11p_sd15_seg.pth", "https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_seg.pth")

#@markdown #### 13. control_v11p_sd15_softedge.pth
#@markdown  画像からソフトエッジを検出するモデルで、検出されたエッジを基に画像の輪郭をぼかしたり、滑らかにしたりする効果を制御することができます。
use_control_v11p_sd15_softedge = False #@param {type: "boolean"}
download_controlnet(use_control_v11p_sd15_softedge, "control_v11p_sd15_softedge.pth", "https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_softedge.pth")

#@markdown #### 14. control_v11p_sd15s2_lineart_anime.pth
#@markdown  画像からアニメ調の線画を抽出するモデルで、抽出された線画を基にアニメ風のイラストを作成するのに適しています。
use_control_v11p_sd15s2_lineart_anime = False #@param {type: "boolean"}
download_controlnet(use_control_v11p_sd15s2_lineart_anime, "control_v11p_sd15s2_lineart_anime.pth", "https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15s2_lineart_anime.pth")

#@markdown #### 15. ioclab_sd15_recolor.safetensors
#@markdown  画像の色を別の画像の色に合わせる（再配色）モデルで、色調やカラースキームを調整するのに利用できます。
use_ioclab_sd15_recolor = False #@param {type: "boolean"}
download_controlnet(use_ioclab_sd15_recolor, "ioclab_sd15_recolor.safetensors", "https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/ioclab_sd15_recolor.safetensors")

#@markdown #### 16. ip-adapter_sd15.pth
#@markdown  画像のスタイルや内容を他の画像から転送するモデルで、画像のスタイルを適用したり、他の画像を参考に画像生成を制御することができます。
use_ip_adapter_sd15 = False #@param {type: "boolean"}
download_controlnet(use_ip_adapter_sd15, "ip-adapter_sd15.pth", "https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/ip-adapter_sd15.pth")

#@markdown #### 17. ip-adapter_sd15_plus.pth
#@markdown  `ip-adapter_sd15.pth` を強化したモデルで、より高度なスタイルの転送や内容の制御が可能です。
use_ip_adapter_sd15_plus = False #@param {type: "boolean"}
download_controlnet(use_ip_adapter_sd15_plus, "ip-adapter_sd15_plus.pth", "https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/ip-adapter_sd15_plus.pth")

#@markdown #### 18. control_sd15_inpaint_depth_hand_fp16.safetensors
#@markdown  画像のインペイント処理に特化し、特に手のような複雑な形状の深度情報を利用してより自然な修復を行うモデルです。
use_control_sd15_inpaint_depth_hand_fp16 = False #@param {type: "boolean"}
download_controlnet(use_control_sd15_inpaint_depth_hand_fp16, "control_sd15_inpaint_depth_hand_fp16.safetensors", "https://huggingface.co/hr16/ControlNet-HandRefiner-pruned/resolve/main/control_sd15_inpaint_depth_hand_fp16.safetensors")

#@markdown #### 19. ip-adapter-faceid_sd15.bin
#@markdown  特定の人物の顔の特徴を学習し、その人物の顔を他の画像に適用するためのモデルです。
use_ip_adapter_faceid_sd15 = False #@param {type: "boolean"}
download_controlnet(use_ip_adapter_faceid_sd15, "ip-adapter-faceid_sd15.bin", "https://huggingface.co/h94/IP-Adapter-FaceID/resolve/main/ip-adapter-faceid_sd15.bin", is_lora=False)

#@markdown #### 20. ip-adapter-faceid-plus_sd15.bin
#@markdown  `ip-adapter-faceid_sd15.bin` を強化したモデルで、より複雑な顔の特徴を学習し、高品質な顔の転送を可能にします。
use_ip_adapter_faceid_plus_sd15 = False #@param {type: "boolean"}
download_controlnet(use_ip_adapter_faceid_plus_sd15, "ip-adapter-faceid-plus_sd15.bin", "https://huggingface.co/h94/IP-Adapter-FaceID/resolve/main/ip-adapter-faceid-plus_sd15.bin", is_lora=False)

#@markdown #### 21. ip-adapter-faceid-plusv2_sd15.bin
#@markdown `ip-adapter-faceid-plus_sd15.bin` をさらに強化したバージョンで、より高度な顔の特徴の学習と転送を可能にします。
use_ip_adapter_faceid_plusv2_sd15 = False #@param {type: "boolean"}
download_controlnet(use_ip_adapter_faceid_plusv2_sd15, "ip-adapter-faceid-plusv2_sd15.bin", "https://huggingface.co/h94/IP-Adapter-FaceID/resolve/main/ip-adapter-faceid-plusv2_sd15.bin", is_lora=False)

#@markdown #### 22. ip-adapter-faceid-portrait_sd15.bin
#@markdown ポートレート写真に特化して学習された `ip-adapter-faceid_sd15` のモデルです。ポートレート写真の顔の特徴をより正確に転送できます。
use_ip_adapter_faceid_portrait_sd15 = False #@param {type: "boolean"}
download_controlnet(use_ip_adapter_faceid_portrait_sd15, "ip-adapter-faceid-portrait_sd15.bin", "https://huggingface.co/h94/IP-Adapter-FaceID/resolve/main/ip-adapter-faceid-portrait_sd15.bin", is_lora=False)

#@markdown #### 23. ip-adapter-faceid_sd15_lora.safetensors
#@markdown `ip-adapter-faceid_sd15.bin` のLoRAモデルで、より軽量で転送性能を高めたモデルです。
use_ip_adapter_faceid_sd15_lora = False #@param {type: "boolean"}
download_controlnet(use_ip_adapter_faceid_sd15_lora, "ip-adapter-faceid_sd15_lora.safetensors", "https://huggingface.co/h94/IP-Adapter-FaceID/resolve/main/ip-adapter-faceid_sd15_lora.safetensors", is_lora=True)

#@markdown #### 24. ip-adapter-faceid-plus_sd15_lora.safetensors
#@markdown  `ip-adapter-faceid-plus_sd15.bin` のLoRAモデルで、より軽量で転送性能を高めたモデルです。
use_ip_adapter_faceid_plus_sd15_lora = False #@param {type: "boolean"}
download_controlnet(use_ip_adapter_faceid_plus_sd15_lora, "ip-adapter-faceid-plus_sd15_lora.safetensors", "https://huggingface.co/h94/IP-Adapter-FaceID/resolve/main/ip-adapter-faceid-plus_sd15_lora.safetensors", is_lora=True)

#@markdown #### 25. ip-adapter-faceid-plusv2_sd15_lora.safetensors
#@markdown `ip-adapter-faceid-plusv2_sd15.bin` のLoRAモデルで、より軽量で転送性能を高めたモデルです。
use_ip_adapter_faceid_plusv2_sd15_lora = False #@param {type: "boolean"}
download_controlnet(use_ip_adapter_faceid_plusv2_sd15_lora, "ip-adapter-faceid-plusv2_sd15_lora.safetensors", "https://huggingface.co/h94/IP-Adapter-FaceID/resolve/main/ip-adapter-faceid-plusv2_sd15_lora.safetensors", is_lora=True)

#@markdown ---
#@markdown ---
#@markdown ### **★ SDXL ControlNet Models（一括インストール）**
#@markdown 以下のモデルは、SDXL のベースモデルで使用できます。
use_control_net_sdxl = False #@param {type: "boolean"}
if use_control_net_sdxl:
    sdxl_model_urls = {
        "diffusers_xl_canny_full.safetensors": "https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/diffusers_xl_canny_full.safetensors",
        "diffusers_xl_canny_mid.safetensors": "https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/diffusers_xl_canny_mid.safetensors",
        "diffusers_xl_canny_small.safetensors": "https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/diffusers_xl_canny_small.safetensors",
        "diffusers_xl_depth_full.safetensors": "https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/diffusers_xl_depth_full.safetensors",
        "diffusers_xl_depth_mid.safetensors": "https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/diffusers_xl_depth_mid.safetensors",
        "diffusers_xl_depth_small.safetensors": "https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/diffusers_xl_depth_small.safetensors",
        "kohya_controllllite_xl_canny_anime.safetensors": "https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/kohya_controllllite_xl_canny_anime.safetensors",
        "kohya_controllllite_xl_depth_anime.safetensors": "https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/kohya_controllllite_xl_depth_anime.safetensors",
        "kohya_controllllite_xl_openpose_anime.safetensors": "https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/kohya_controllllite_xl_openpose_anime.safetensors",
        "kohya_controllllite_xl_openpose_anime_v2.safetensors": "https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/kohya_controllllite_xl_openpose_anime_v2.safetensors",
        "kohya_controllllite_xl_scribble_anime.safetensors": "https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/kohya_controllllite_xl_scribble_anime.safetensors",
        "sai_xl_canny_128lora.safetensors": "https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/sai_xl_canny_128lora.safetensors",
        "sai_xl_canny_256lora.safetensors": "https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/sai_xl_canny_256lora.safetensors",
        "sai_xl_depth_128lora.safetensors": "https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/sai_xl_depth_128lora.safetensors",
        "sai_xl_depth_256lora.safetensors": "https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/sai_xl_depth_256lora.safetensors",
        "sai_xl_recolor_128lora.safetensors": "https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/sai_xl_recolor_128lora.safetensors",
        "sai_xl_recolor_256lora.safetensors": "https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/sai_xl_recolor_256lora.safetensors",
        "sai_xl_sketch_128lora.safetensors": "https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/sai_xl_sketch_128lora.safetensors",
        "sai_xl_sketch_256lora.safetensors": "https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/sai_xl_sketch_256lora.safetensors",
        "sargezt_xl_depth.safetensors": "https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/sargezt_xl_depth.safetensors",
        "sargezt_xl_depth_faid_vidit.safetensors": "https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/sargezt_xl_depth_faid_vidit.safetensors",
        "sargezt_xl_depth_zeed.safetensors": "https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/sargezt_xl_depth_zeed.safetensors",
        "sargezt_xl_softedge.safetensors": "https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/sargezt_xl_softedge.safetensors",
        "t2i-adapter_xl_canny.safetensors": "https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/t2i-adapter_xl_canny.safetensors",
        "t2i-adapter_xl_openpose.safetensors": "https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/t2i-adapter_xl_openpose.safetensors",
        "t2i-adapter_xl_sketch.safetensors": "https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/t2i-adapter_xl_sketch.safetensors",
        "thibaud_xl_openpose.safetensors": "https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/thibaud_xl_openpose.safetensors",
        "thibaud_xl_openpose_256lora.safetensors": "https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/thibaud_xl_openpose_256lora.safetensors",
        "ip-adapter_xl.pth": "https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/ip-adapter_xl.pth",
    }

    for filename, url in sdxl_model_urls.items():
      download_controlnet(use_control_net_sdxl, filename, url)
    # IP-Adapter-FaceID
    face_id_xl_model_urls = {
        "ip-adapter-faceid_sdxl.bin": "https://huggingface.co/h94/IP-Adapter-FaceID/resolve/main/ip-adapter-faceid_sdxl.bin",
        "ip-adapter-faceid-plusv2_sdxl.bin": "https://huggingface.co/h94/IP-Adapter-FaceID/resolve/main/ip-adapter-faceid-plusv2_sdxl.bin",
        "ip-adapter-faceid_sdxl_lora.safetensors": "https://huggingface.co/h94/IP-Adapter-FaceID/resolve/main/ip-adapter-faceid_sdxl_lora.safetensors",
        "ip-adapter-faceid-plusv2_sdxl_lora.safetensors": "https://huggingface.co/h94/IP-Adapter-FaceID/resolve/main/ip-adapter-faceid-plusv2_sdxl_lora.safetensors",
    }
    for filename, url in face_id_xl_model_urls.items():
        download_controlnet(use_control_net_sdxl, filename, url, is_lora=filename.endswith(".safetensors"))
    # Instant ID
    instant_id_model_urls = {
        "ip-adapter_instant_id_sdxl.bin": "https://huggingface.co/InstantX/InstantID/resolve/main/ip-adapter.bin",
        "control_instant_id_sdxl.safetensors": "https://huggingface.co/InstantX/InstantID/resolve/main/ControlNetModel/diffusion_pytorch_model.safetensors"
    }
    for filename, url in instant_id_model_urls.items():
        download_controlnet(use_control_net_sdxl, filename, url)

#@markdown ---
#@markdown ### **★ SDXL ControlNet Models（個別に選択してインストール）**
#@markdown 以下のモデルは、SDXL のベースモデルで使用できます。

#@markdown #### 1. diffusers_xl_canny_full.safetensors
#@markdown SDXL用に調整されたCannyエッジ検出モデルで、画像内のエッジを捉え、そのエッジ情報に基づいて画像を生成します。
use_diffusers_xl_canny_full = False #@param {type: "boolean"}
download_controlnet(use_diffusers_xl_canny_full, "diffusers_xl_canny_full.safetensors", "https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/diffusers_xl_canny_full.safetensors")

#@markdown #### 2. diffusers_xl_canny_mid.safetensors
#@markdown  `diffusers_xl_canny_full.safetensors` の中間的な解像度に対応したモデルで、処理速度と精度をバランスさせたエッジ検出を行います。
use_diffusers_xl_canny_mid = False #@param {type: "boolean"}
download_controlnet(use_diffusers_xl_canny_mid, "diffusers_xl_canny_mid.safetensors", "https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/diffusers_xl_canny_mid.safetensors")

#@markdown #### 3. diffusers_xl_canny_small.safetensors
#@markdown  `diffusers_xl_canny_full.safetensors` の低解像度版で、高速な処理に適していますが、エッジ検出の精度はやや劣ります。
use_diffusers_xl_canny_small = False #@param {type: "boolean"}
download_controlnet(use_diffusers_xl_canny_small, "diffusers_xl_canny_small.safetensors", "https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/diffusers_xl_canny_small.safetensors")

#@markdown #### 4. diffusers_xl_depth_full.safetensors
#@markdown SDXL用に調整された深度マップ生成モデルで、画像の奥行き情報を捉え、3Dのような効果を生成するのに使用します。
use_diffusers_xl_depth_full = False #@param {type: "boolean"}
download_controlnet(use_diffusers_xl_depth_full, "diffusers_xl_depth_full.safetensors", "https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/diffusers_xl_depth_full.safetensors")

#@markdown #### 5. diffusers_xl_depth_mid.safetensors
#@markdown `diffusers_xl_depth_full.safetensors` の中間的な解像度に対応したモデルで、処理速度と精度をバランスさせた深度マップ生成を行います。
use_diffusers_xl_depth_mid = False #@param {type: "boolean"}
download_controlnet(use_diffusers_xl_depth_mid, "diffusers_xl_depth_mid.safetensors", "https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/diffusers_xl_depth_mid.safetensors")

#@markdown #### 6. diffusers_xl_depth_small.safetensors
#@markdown `diffusers_xl_depth_full.safetensors` の低解像度版で、高速な処理に適していますが、深度マップの精度はやや劣ります。
use_diffusers_xl_depth_small = False #@param {type: "boolean"}
download_controlnet(use_diffusers_xl_depth_small, "diffusers_xl_depth_small.safetensors", "https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/diffusers_xl_depth_small.safetensors")

#@markdown #### 7. kohya_controllllite_xl_canny_anime.safetensors
#@markdown アニメ調の画像に特化したCannyエッジ検出モデルで、アニメスタイルの線画をより正確に捉えます。
use_kohya_controllllite_xl_canny_anime = False #@param {type: "boolean"}
download_controlnet(use_kohya_controllllite_xl_canny_anime, "kohya_controllllite_xl_canny_anime.safetensors", "https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/kohya_controllllite_xl_canny_anime.safetensors")

#@markdown #### 8. kohya_controllllite_xl_depth_anime.safetensors
#@markdown  アニメ調の画像に特化した深度マップ生成モデルで、アニメ特有の奥行き表現を捉えます。
use_kohya_controllllite_xl_depth_anime = False #@param {type: "boolean"}
download_controlnet(use_kohya_controllllite_xl_depth_anime, "kohya_controllllite_xl_depth_anime.safetensors", "https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/kohya_controllllite_xl_depth_anime.safetensors")

#@markdown #### 9. kohya_controllllite_xl_openpose_anime.safetensors
#@markdown アニメ調の画像に特化した人物のポーズ検出モデルで、アニメキャラのポーズをより正確に検出します。
use_kohya_controllllite_xl_openpose_anime = False #@param {type: "boolean"}
download_controlnet(use_kohya_controllllite_xl_openpose_anime, "kohya_controllllite_xl_openpose_anime.safetensors", "https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/kohya_controllllite_xl_openpose_anime.safetensors")

#@markdown #### 10. kohya_controllllite_xl_openpose_anime_v2.safetensors
#@markdown  `kohya_controllllite_xl_openpose_anime.safetensors` の改良版で、検出精度と安定性が向上しています。
use_kohya_controllllite_xl_openpose_anime_v2 = False #@param {type: "boolean"}
download_controlnet(use_kohya_controllllite_xl_openpose_anime_v2, "kohya_controllllite_xl_openpose_anime_v2.safetensors", "https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/kohya_controllllite_xl_openpose_anime_v2.safetensors")

#@markdown #### 11. kohya_controllllite_xl_scribble_anime.safetensors
#@markdown アニメ調のラフスケッチを基に画像を生成するモデルで、手書き風の線画を捉えるのに適しています。
use_kohya_controllllite_xl_scribble_anime = False #@param {type: "boolean"}
download_controlnet(use_kohya_controllllite_xl_scribble_anime, "kohya_controllllite_xl_scribble_anime.safetensors", "https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/kohya_controllllite_xl_scribble_anime.safetensors")

#@markdown #### 12. sai_xl_canny_128lora.safetensors
#@markdown SDXL用のCannyエッジ検出モデルのLoRA版で、128解像度に対応しており、軽量で高速な処理が可能です。
use_sai_xl_canny_128lora = False #@param {type: "boolean"}
download_controlnet(use_sai_xl_canny_128lora, "sai_xl_canny_128lora.safetensors", "https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/sai_xl_canny_128lora.safetensors", is_lora=True)

#@markdown #### 13. sai_xl_canny_256lora.safetensors
#@markdown SDXL用のCannyエッジ検出モデルのLoRA版で、256解像度に対応しており、`sai_xl_canny_128lora.safetensors`よりも高精度なエッジ検出を行います。
use_sai_xl_canny_256lora = False #@param {type: "boolean"}
download_controlnet(use_sai_xl_canny_256lora, "sai_xl_canny_256lora.safetensors", "https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/sai_xl_canny_256lora.safetensors", is_lora=True)

#@markdown #### 14. sai_xl_depth_128lora.safetensors
#@markdown  SDXL用の深度マップ生成モデルのLoRA版で、128解像度に対応しており、軽量で高速な処理が可能です。
use_sai_xl_depth_128lora = False #@param {type: "boolean"}
download_controlnet(use_sai_xl_depth_128lora, "sai_xl_depth_128lora.safetensors", "https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/sai_xl_depth_128lora.safetensors", is_lora=True)

#@markdown #### 15. sai_xl_depth_256lora.safetensors
#@markdown SDXL用の深度マップ生成モデルのLoRA版で、256解像度に対応しており、`sai_xl_depth_128lora.safetensors`よりも高精度な深度情報を生成します。
use_sai_xl_depth_256lora = False #@param {type: "boolean"}
download_controlnet(use_sai_xl_depth_256lora, "sai_xl_depth_256lora.safetensors", "https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/sai_xl_depth_256lora.safetensors", is_lora=True)

#@markdown #### 16. sai_xl_recolor_128lora.safetensors
#@markdown SDXL用の再配色モデルのLoRA版で、128解像度に対応しており、軽量で高速な処理が可能です。
use_sai_xl_recolor_128lora = False #@param {type: "boolean"}
download_controlnet(use_sai_xl_recolor_128lora, "sai_xl_recolor_128lora.safetensors", "https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/sai_xl_recolor_128lora.safetensors", is_lora=True)

#@markdown #### 17. sai_xl_recolor_256lora.safetensors
#@markdown  SDXL用の再配色モデルのLoRA版で、256解像度に対応しており、`sai_xl_recolor_128lora.safetensors`よりも高精度な色調整が可能です。
use_sai_xl_recolor_256lora = False #@param {type: "boolean"}
download_controlnet(use_sai_xl_recolor_256lora, "sai_xl_recolor_256lora.safetensors", "https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/sai_xl_recolor_256lora.safetensors", is_lora=True)

#@markdown #### 18. sai_xl_sketch_128lora.safetensors
#@markdown SDXL用のスケッチモデルのLoRA版で、128解像度に対応しており、軽量で高速な処理が可能です。
use_sai_xl_sketch_128lora = False #@param {type: "boolean"}
download_controlnet(use_sai_xl_sketch_128lora, "sai_xl_sketch_128lora.safetensors", "https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/sai_xl_sketch_128lora.safetensors", is_lora=True)

#@markdown #### 19. sai_xl_sketch_256lora.safetensors
#@markdown SDXL用のスケッチモデルのLoRA版で、256解像度に対応しており、`sai_xl_sketch_128lora.safetensors`よりも高精度なスケッチの抽出が可能です。
use_sai_xl_sketch_256lora = False #@param {type: "boolean"}
download_controlnet(use_sai_xl_sketch_256lora, "sai_xl_sketch_256lora.safetensors", "https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/sai_xl_sketch_256lora.safetensors", is_lora=True)

#@markdown #### 20. sargezt_xl_depth.safetensors
#@markdown  SDXL用の深度マップ生成モデルで、`diffusers_xl_depth`系モデルとは異なるアルゴリズムで深度マップを生成します。
use_sargezt_xl_depth = False #@param {type: "boolean"}
download_controlnet(use_sargezt_xl_depth, "sargezt_xl_depth.safetensors", "https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/sargezt_xl_depth.safetensors")

#@markdown #### 21. sargezt_xl_depth_faid_vidit.safetensors
#@markdown  `sargezt_xl_depth.safetensors` を基にした派生モデルで、より多様な深度表現に対応します。
use_sargezt_xl_depth_faid_vidit = False #@param {type: "boolean"}
download_controlnet(use_sargezt_xl_depth_faid_vidit, "sargezt_xl_depth_faid_vidit.safetensors", "https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/sargezt_xl_depth_faid_vidit.safetensors")

#@markdown #### 22. sargezt_xl_depth_zeed.safetensors
#@markdown  `sargezt_xl_depth.safetensors` を基にした派生モデルで、特定の条件下での深度表現を向上させています。
use_sargezt_xl_depth_zeed = False #@param {type: "boolean"}
download_controlnet(use_sargezt_xl_depth_zeed, "sargezt_xl_depth_zeed.safetensors", "https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/sargezt_xl_depth_zeed.safetensors")

#@markdown #### 23. sargezt_xl_softedge.safetensors
#@markdown  SDXL用のソフトエッジ検出モデルで、画像の輪郭をぼかすために使用します。
use_sargezt_xl_softedge = False #@param {type: "boolean"}
download_controlnet(use_sargezt_xl_softedge, "sargezt_xl_softedge.safetensors", "https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/sargezt_xl_softedge.safetensors")

#@markdown #### 24. t2i-adapter_xl_canny.safetensors
#@markdown  SDXL用のCannyエッジ検出モデルで、`diffusers_xl_canny`系とは異なるアルゴリズムでエッジを検出します。
use_t2i_adapter_xl_canny = False #@param {type: "boolean"}
download_controlnet(use_t2i_adapter_xl_canny, "t2i-adapter_xl_canny.safetensors", "https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/t2i-adapter_xl_canny.safetensors")

#@markdown #### 25. t2i-adapter_xl_openpose.safetensors
#@markdown  SDXL用の人物ポーズ検出モデルで、`kohya_controllllite_xl_openpose_anime`系とは異なるアルゴリズムでポーズを検出します。
use_t2i_adapter_xl_openpose = False #@param {type: "boolean"}
download_controlnet(use_t2i_adapter_xl_openpose, "t2i-adapter_xl_openpose.safetensors", "https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/t2i-adapter_xl_openpose.safetensors")

#@markdown #### 26. t2i-adapter_xl_sketch.safetensors
#@markdown  SDXL用のスケッチモデルで、`sai_xl_sketch`系とは異なるアルゴリズムでスケッチを抽出します。
use_t2i_adapter_xl_sketch = False #@param {type: "boolean"}
download_controlnet(use_t2i_adapter_xl_sketch, "t2i-adapter_xl_sketch.safetensors", "https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/t2i-adapter_xl_sketch.safetensors")

#@markdown #### 27. thibaud_xl_openpose.safetensors
#@markdown  SDXL用の人物ポーズ検出モデルで、`kohya_controllllite_xl_openpose_anime`系や`t2i-adapter_xl_openpose`とは異なるアルゴリズムでポーズを検出します。
use_thibaud_xl_openpose = False #@param {type: "boolean"}
download_controlnet(use_thibaud_xl_openpose, "thibaud_xl_openpose.safetensors", "https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/thibaud_xl_openpose.safetensors")

#@markdown #### 28. thibaud_xl_openpose_256lora.safetensors
#@markdown `thibaud_xl_openpose.safetensors` のLoRA版で、256解像度に対応しており、高精度なポーズ検出が可能です。
use_thibaud_xl_openpose_256lora = False #@param {type: "boolean"}
download_controlnet(use_thibaud_xl_openpose_256lora, "thibaud_xl_openpose_256lora.safetensors", "https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/thibaud_xl_openpose_256lora.safetensors", is_lora=True)

#@markdown #### 29. ip-adapter_xl.pth
#@markdown SDXL用のIP-Adapterモデルで、画像からスタイルや内容を転送するモデルです。
use_ip_adapter_xl = False #@param {type: "boolean"}
download_controlnet(use_ip_adapter_xl, "ip-adapter_xl.pth", "https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/ip-adapter_xl.pth")

#@markdown #### 30. ip-adapter-faceid_sdxl.bin
#@markdown  特定の人物の顔の特徴を学習し、SDXLでその人物の顔を他の画像に適用するためのモデルです。
use_ip_adapter_faceid_sdxl = False #@param {type: "boolean"}
download_controlnet(use_ip_adapter_faceid_sdxl, "ip-adapter-faceid_sdxl.bin", "https://huggingface.co/h94/IP-Adapter-FaceID/resolve/main/ip-adapter-faceid_sdxl.bin")

#@markdown #### 31. ip-adapter-faceid-plusv2_sdxl.bin
#@markdown `ip-adapter-faceid_sdxl.bin` を強化したモデルで、より高度な顔の特徴の学習と転送を可能にします。
use_ip_adapter_faceid_plusv2_sdxl = False #@param {type: "boolean"}
download_controlnet(use_ip_adapter_faceid_plusv2_sdxl, "ip-adapter-faceid-plusv2_sdxl.bin", "https://huggingface.co/h94/IP-Adapter-FaceID/resolve/main/ip-adapter-faceid-plusv2_sdxl.bin")

#@markdown #### 32. ip-adapter-faceid_sdxl_lora.safetensors
#@markdown  `ip-adapter-faceid_sdxl.bin` のLoRAモデルで、より軽量で転送性能を高めたモデルです。
use_ip_adapter_faceid_sdxl_lora = False #@param {type: "boolean"}
download_controlnet(use_ip_adapter_faceid_sdxl_lora, "ip-adapter-faceid_sdxl_lora.safetensors", "https://huggingface.co/h94/IP-Adapter-FaceID/resolve/main/ip-adapter-faceid_sdxl_lora.safetensors", is_lora=True)

#@markdown #### 33. ip-adapter-faceid-plusv2_sdxl_lora.safetensors
#@markdown  `ip-adapter-faceid-plusv2_sdxl.bin` のLoRAモデルで、より軽量で転送性能を高めたモデルです。
use_ip_adapter_faceid_plusv2_sdxl_lora = False #@param {type: "boolean"}
download_controlnet(use_ip_adapter_faceid_plusv2_sdxl_lora, "ip-adapter-faceid-plusv2_sdxl_lora.safetensors", "https://huggingface.co/h94/IP-Adapter-FaceID/resolve/main/ip-adapter-faceid-plusv2_sdxl_lora.safetensors", is_lora=True)

#@markdown #### 34. ip-adapter_instant_id_sdxl.bin
#@markdown  Instant IDに特化したSDXL用のIP-Adapterモデルです。
use_ip_adapter_instant_id_sdxl = False #@param {type: "boolean"}
download_controlnet(use_ip_adapter_instant_id_sdxl, "ip-adapter_instant_id_sdxl.bin", "https://huggingface.co/InstantX/InstantID/resolve/main/ip-adapter.bin")

#@markdown #### 35. control_instant_id_sdxl.safetensors
#@markdown  Instant IDに特化したSDXL用のControlNetモデルです。
use_control_instant_id_sdxl = False #@param {type: "boolean"}
download_controlnet(use_control_instant_id_sdxl, "control_instant_id_sdxl.safetensors", "https://huggingface.co/InstantX/InstantID/resolve/main/ControlNetModel/diffusion_pytorch_model.safetensors")

end_time = time.time()
elapsed_time = end_time - start_time
print(f"すべての処理が完全に完了しました！({elapsed_time:.2f}秒)")

## 8.select Extensions

In [ ]:
#@title ### **For Extension**

#@markdown ## **Extension**
#@markdown ---
#@markdown ※導入したいExtensionにチェックを入れてください

# --- 設定 ---
# Stable Diffusion Web UI のディレクトリ
extension_dir = "/content/stable-diffusion-webui/extensions"


# 拡張機能をインストール（直接git clone）
print("Extensionのダウンロードを開始します...")
start_time = time.time()

#@markdown ### ★ [After Detailer](https://github.com/Bing-su/adetailer)
#@markdown  生成された画像の顔や手などの細かい部分を、追加で高画質化・修正する拡張機能です。
#@markdown [使い方を確認する(YouTube)](https://youtu.be/WJfhNjzjMIE)
use_after_detailer = False #@param {type: "boolean"}
if use_after_detailer:
  !git clone https://github.com/Bing-su/adetailer.git {extension_dir}/adetailer
  # ADetailer (After Detailer) Lips Model (URL: https://civitai.com/models/142240/adetailer-after-detailer-lips-model)
  !mkdir -p /content/stable-diffusion-webui/models/adetailer
  !wget https://civitai.com/api/download/models/157700?token={api_key_civitai} --content-disposition --directory-prefix=/content/stable-diffusion-webui/models/adetailer
  !unzip /content/stable-diffusion-webui/models/adetailer/adetailerAfterDetailer_v10.zip -d /content/stable-diffusion-webui/models/adetailer/

#@markdown ### ★ [AnimateDiff](https://github.com/continue-revolution/sd-webui-animatediff)
#@markdown  Stable DiffusionでアニメーションGIFや動画を生成できるようにする拡張機能です。
#@markdown [使い方を確認する(YouTube)](https://youtu.be/jzt8LXczMb4)
use_animate_diff = False #@param {type: "boolean"}
if use_animate_diff:
  # AnimateDiffの導入
  !git clone https://github.com/continue-revolution/sd-webui-animatediff.git {extension_dir}/sd-webui-animatediff

#@markdown 以下にチェックをつけると、AnimateDiff用のmodelはv15_v3のみをダウンロードするようにします。チェックをつけたほうがセットアップ処理の時間削減に繋がります。（v15_v2のみのチェックボックスと併用可能です）

#@markdown ※AnimateDiffで利用するモーションモジュールをv15_v3のみに限定し、ダウンロード時間を短縮するための設定です。
only_use_animate_diff_v15_v3 = False #@param {type: "boolean"}
if only_use_animate_diff_v15_v3:
  # AnimateDiff用のモーションモジュールダウンロード (v15_v3)
  !wget https://huggingface.co/conrevo/AnimateDiff-A1111/resolve/main/motion_module/mm_sd15_v3.safetensors --directory-prefix={extension_dir}/sd-webui-animatediff/model

#@markdown 以下にチェックをつけると、AnimateDiff用のmodelはv15_v2のみをダウンロードするようにします。チェックをつけたほうがセットアップ処理の時間削減に繋がります。（v15_v3のみのチェックボックスと併用可能です）

#@markdown ※AnimateDiffで利用するモーションモジュールをv15_v2のみに限定し、ダウンロード時間を短縮するための設定です。
only_use_animate_diff_v15_v2 = False #@param {type: "boolean"}
if only_use_animate_diff_v15_v2:
  # AnimateDiff用のモーションモジュールダウンロード (v15_v2)
  !wget https://huggingface.co/guoyww/animatediff/resolve/refs%2Fpr%2F3/mm_sd_v15_v2.safetensors --directory-prefix=/content/stable-diffusion-webui/extensions/sd-webui-animatediff/model

if only_use_animate_diff_v15_v3 is False and only_use_animate_diff_v15_v2 is False and use_animate_diff is True:
  !wget https://huggingface.co/guoyww/animatediff/resolve/refs%2Fpr%2F3/mm_sd_v14.safetensors --directory-prefix=/content/stable-diffusion-webui/extensions/sd-webui-animatediff/model
  !wget https://huggingface.co/guoyww/animatediff/resolve/refs%2Fpr%2F3/mm_sd_v15.safetensors --directory-prefix=/content/stable-diffusion-webui/extensions/sd-webui-animatediff/model
  !wget https://huggingface.co/guoyww/animatediff/resolve/refs%2Fpr%2F3/mm_sd_v15_v2.safetensors --directory-prefix=/content/stable-diffusion-webui/extensions/sd-webui-animatediff/model
  !wget https://huggingface.co/conrevo/AnimateDiff-A1111/resolve/main/motion_module/mm_sd15_v3.safetensors --directory-prefix=/content/stable-diffusion-webui/extensions/sd-webui-animatediff/model

#@markdown ### ★ [Booru tag autocompletion for A1111](https://github.com/DominikDoom/a1111-sd-webui-tagcomplete)
#@markdown  画像生成時に、Booru系のサイト（danbooruなど）で使用されるタグの入力を補助する機能を追加する拡張機能です。
use_booru_tag_autocompletion_for_a1111 = False #@param {type: "boolean"}
if use_booru_tag_autocompletion_for_a1111:
  !git clone https://github.com/DominikDoom/a1111-sd-webui-tagcomplete.git {extension_dir}/tag-autocomplete

#@markdown ### ★ [ControlNet for Stable Diffusion WebUI](https://github.com/Mikubill/sd-webui-controlnet) ※上記のControl Net用のコードでダウンロード済みのためここでは処理しない
#@markdown ※生成する画像のポーズや構図を指定できる
#@markdown  生成する画像の構図、ポーズ、スタイルを、別の画像やスケッチなどを使って細かく制御できるようにする拡張機能です。
#@markdown [使い方を確認する(YouTube)](https://)
# use_Control_Net = True #@param {type: "boolean"}
# if use_Control_Net:
#   !git clone https://github.com/Mikubill/sd-webui-controlnet.git {extension_dir}/sd-webui-controlnet

#@markdown ### ★ [Deforum Stable Diffusion — official extension for AUTOMATIC1111's webui](https://github.com/deforum-art/sd-webui-deforum)
#@markdown  動画生成に特化した拡張機能で、パラメータをアニメーションさせることで、複雑な動く画像を生成できます。
use_sd_webui_deforum = False #@param {type: "boolean"}
if use_sd_webui_deforum:
  !git clone https://github.com/deforum-art/sd-webui-deforum.git /content/stable-diffusion-webui/extensions/deforum

#@markdown ### ★ [Detail Daemon](https://github.com/muerrilla/sd-webui-detail-daemon/)
#@markdown  画像を生成する際に、設定した条件で自動的に詳細化（アップスケール）を行う拡張機能です。
use_detail_daemon = False #@param {type: "boolean"}
if use_detail_daemon:
  !git clone https://github.com/muerrilla/sd-webui-detail-daemon.git /content/stable-diffusion-webui/extensions/sd-webui-detail-daemon

#@markdown ### ★ [Dataset Tag Editor](https://github.com/toshiaki1729/stable-diffusion-webui-dataset-tag-editor)
#@markdown  Stable Diffusionの学習用データセットのタグ編集を効率化する拡張機能です。
use_dataset_tag_editor = False #@param {type: "boolean"}
if use_dataset_tag_editor:
  !git clone https://github.com/toshiaki1729/stable-diffusion-webui-dataset-tag-editor.git /content/stable-diffusion-webui/extensions/stable-diffusion-webui-dataset-tag-editor

#@markdown ### ★ [Easy Generate Forever](https://github.com/blue-pen5805/sdweb-easy-generate-forever)
#@markdown 指定した設定で画像を無限に生成し続けるための拡張機能です。
#@markdown [使い方を確認する(YouTube)](https://youtu.be/e_LTlWbKTaE)
use_easy_generate_forever = False #@param {type: "boolean"}
if use_easy_generate_forever:
  !git clone https://github.com/blue-pen5805/sdweb-easy-generate-forever.git {extension_dir}/sdweb-easy-generate-forever

#@markdown ### ★ [Inpaint Anything for Stable Diffusion Web UI](https://github.com/Uminosachi/sd-webui-inpaint-anything)
#@markdown  画像内の特定の領域を、マスクで指定して修復（インペイント）できる拡張機能です。
use_sd_webui_inpaint_anything = False #@param {type: "boolean"}
if use_sd_webui_inpaint_anything:
  # Inpaint Anything for Stable Diffusion Web UI
  !git clone https://github.com/Uminosachi/sd-webui-inpaint-anything.git /content/stable-diffusion-webui/extensions/sd-webui-inpaint-anything

#@markdown ### ★ [LoRA Block Weight](https://github.com/hako-mikan/sd-webui-lora-block-weight)
#@markdown  LoRAモデルをより細かく制御するための拡張機能で、LoRAの各ブロックに異なる重みを設定できます。
use_lora_block_weight = False #@param {type: "boolean"}
if use_lora_block_weight:
  !git clone https://github.com/hako-mikan/sd-webui-lora-block-weight.git /content/stable-diffusion-webui/extensions/sd-webui-lora-block-weight

#@markdown ### ★ [Mov2mov](https://github.com/Scholar01/sd-webui-mov2mov)
#@markdown  動画を基に新たな動画を生成したり、動画のスタイルを変更したりするのに使用できる拡張機能です。
use_mov2mov = False #@param {type: "boolean"}
if use_mov2mov:
  !git clone https://github.com/Scholar01/sd-webui-mov2mov.git /content/stable-diffusion-webui/extensions/sd-webui-mov2mov

#@markdown ### ★ [ReActor for Stable Diffusion](https://github.com/Gourieff/sd-webui-reactor)
#@markdown  画像内の人物の顔を別の顔に置き換える（顔交換）するための拡張機能です。
use_sd_webui_reactor = False #@param {type: "boolean"}
if use_sd_webui_reactor:
  !git clone https://github.com/Gourieff/sd-webui-reactor.git /content/stable-diffusion-webui/extensions/sd-webui-reactor

#@markdown ### ★ [Rembg](https://github.com/AUTOMATIC1111/stable-diffusion-webui-rembg)
#@markdown  画像の背景を自動的に削除（透過）する拡張機能です。
#@markdown [使い方を確認する(YouTube)](https://youtu.be/nK9M7NYgZDI)
use_rembg = False #@param {type: "boolean"}
if use_rembg:
  !git clone https://github.com/AUTOMATIC1111/stable-diffusion-webui-rembg.git {extension_dir}/stable-diffusion-webui-rembg

#@markdown ### ★ [sd-civitai-browser (CivBrowser)](https://github.com/SignalFlagZ/sd-webui-civbrowser)
#@markdown  CivitaiのモデルやLoRAなどをweb UI上から検索・ダウンロードできる拡張機能です。
use_sd_webui_civbrowser = False #@param {type: "boolean"}
if use_sd_webui_civbrowser:
  !git clone https://github.com/SignalFlagZ/sd-webui-civbrowser.git /content/stable-diffusion-webui/extensions/sd-webui-civbrowser

#@markdown ### ★ [sd_katanuki](https://github.com/aka7774/sd_katanuki)
#@markdown  画像内のオブジェクトを切り抜くための拡張機能で、より正確なマスク作成をサポートします。
#@markdown [使い方を確認する(YouTube)](https://youtu.be/3WiiXMWierk)
use_sd_katanuki = False #@param {type: "boolean"}
if use_sd_katanuki:
  !git clone https://github.com/aka7774/sd_katanuki.git {extension_dir}/sd_katanuki

#@markdown ### ★ [sd-webui-depth-lib](https://github.com/jexom/sd-webui-depth-lib) ※手の修正
#@markdown 画像の深度（奥行き）情報を利用して、手の修正などを行うための拡張機能です。
#@markdown [使い方を確認する(YouTube)](https://)
use_sd_webui_depth_lib = False #@param {type: "boolean"}
if use_sd_webui_depth_lib:
  !git clone https://github.com/jexom/sd-webui-depth-lib.git {extension_dir}/sd-webui-depth-lib

#@markdown ### ★ [sd-webui-enable-checker](https://github.com/shirayu/sd-webui-enable-checker)
#@markdown  Stable Diffusionのモデル、VAE、LoRAの有効/無効の状態を管理するための拡張機能です。
#@markdown [使い方を確認する(YouTube)](https://youtu.be/e_LTlWbKTaE)
use_sd_webui_enable_chacker = False #@param {type: "boolean"}
if use_sd_webui_enable_chacker:
  !git clone https://github.com/shirayu/sd-webui-enable-checker.git {extension_dir}/sd-webui-enable-checker

#@markdown ### ★ [sd-webui-freeu](https://github.com/ljleb/sd-webui-freeu)
#@markdown  Stable Diffusionの生成品質を向上させるための手法であるFreeUを実装した拡張機能です。
use_sd_webui_freeu = False #@param {type: "boolean"}
if use_sd_webui_freeu:
  !git clone https://github.com/ljleb/sd-webui-freeu.git /content/stable-diffusion-webui/extensions/sd-webui-freeu

#@markdown ### ★ [SearchMyPNG](https://github.com/hina-choco/SearchMyPNG)
#@markdown  PNG画像に埋め込まれた生成情報などを検索・閲覧する拡張機能です。
use_search_my_png = False #@param {type: "boolean"}
if use_search_my_png:
  !git clone https://github.com/hina-choco/SearchMyPNG.git /content/stable-diffusion-webui/extensions/SearchMyPNG

#@markdown ### ★ [Segment Anything for Stable Diffusion WebUI](https://github.com/continue-revolution/sd-webui-segment-anything)
#@markdown  Meta AIのSegment Anythingモデルを使用し、画像内のオブジェクトを簡単に選択・マスクできる拡張機能です。
use_sd_webui_segment_anything = False #@param {type: "boolean"}
if use_sd_webui_segment_anything:
  # Segment Anything for Stable Diffusion WebUI
  !git clone https://github.com/continue-revolution/sd-webui-segment-anything.git /content/stable-diffusion-webui/extensions/sd-webui-segment-anything
  # 必要なモデルファイルをダウンロード - Meta AI (Colab利用前提のためhuge, largeのみダウンロード)
  !wget https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth --directory-prefix=/content/stable-diffusion-webui/extensions/sd-webui-segment-anything/models/sam
  !wget https://dl.fbaipublicfiles.com/segment_anything/sam_vit_l_0b3195.pth --directory-prefix=/content/stable-diffusion-webui/extensions/sd-webui-segment-anything/models/sam

#@markdown ### ★ [Stable Diffusion webui Infinite Image Browsing](https://github.com/zanllp/sd-webui-infinite-image-browsing)
#@markdown  生成された画像を無限にブラウズできる機能を追加する拡張機能です。
use_sd_webui_infinite_image_browsing = False #@param {type: "boolean"}
if use_sd_webui_infinite_image_browsing:
  !git clone https://github.com/zanllp/sd-webui-infinite-image-browsing.git /content/stable-diffusion-webui/extensions/sd-webui-infinite-image-browsing

#@markdown ### ★ [SuperMerger](https://github.com/hako-mikan/sd-webui-supermerger)
#@markdown  Stable Diffusionのモデルをマージ（合成）するための拡張機能で、新しいモデルを作成できます。
use_super_merger = False #@param {type: "boolean"}
if use_super_merger:
  !git clone https://github.com/hako-mikan/sd-webui-supermerger.git /content/stable-diffusion-webui/extensions/sd-webui-supermerger

#@markdown ### ★ [TrainTrain](https://github.com/hako-mikan/sd-webui-traintrain)
#@markdown  Stable DiffusionのLoRAモデルやEmbeddingなどを学習させるための拡張機能です。
use_traintrain = False #@param {type: "boolean"}
if use_traintrain:
  !git clone https://github.com/hako-mikan/sd-webui-traintrain.git /content/stable-diffusion-webui/extensions/sd-webui-traintrain

#@markdown ### ★ [Manga Editor Desu! Pro Edition](https://github.com/new-sankaku/stable-diffusion-webui-simple-manga-maker)
#@markdown  Stable Diffusionで漫画のような画像を生成するための拡張機能で、コマ割りや吹き出しなどの機能を利用できます。
use_manga_editor_desu_pro_edition = False #@param {type: "boolean"}
if use_manga_editor_desu_pro_edition:
  !git clone https://github.com/new-sankaku/stable-diffusion-webui-simple-manga-maker.git /content/stable-diffusion-webui/extensions/stable-diffusion-webui-simple-manga-maker

print("Extensionのダウンロードが完了しました。")


# Extension（拡張機能）の情報
extension_choices = {
    "After Detailer":{
         "requirements_path": f"{extension_dir}/adetailer/requirements.txt"
    },
    "AnimateDiff":{
      "requirements_path": f"{extension_dir}/sd-webui-animatediff/requirements.txt"
    },
    "Booru tag autocompletion for A1111":{
          "requirements_path": f"{extension_dir}/tag-autocomplete/requirements.txt"
    },
    "sd-webui-controlnet":{
         "requirements_path": f"{extension_dir}/sd-webui-controlnet/requirements.txt"
    },
     "Deforum Stable Diffusion — official extension for AUTOMATIC1111's webui":{
          "requirements_path": f"{extension_dir}/deforum/requirements.txt"
    },
    "Detail Daemon":{
        "requirements_path": f"{extension_dir}/sd-webui-detail-daemon/requirements.txt"
    },
     "Dataset Tag Editor":{
          "requirements_path": f"{extension_dir}/stable-diffusion-webui-dataset-tag-editor/requirements.txt"
     },
     "Easy Generate Forever":{
          "requirements_path": f"{extension_dir}/sdweb-easy-generate-forever/requirements.txt"
    },
    "Inpaint Anything for Stable Diffusion Web UI":{
          "requirements_path": f"{extension_dir}/sd-webui-inpaint-anything/requirements.txt"
    },
    "LoRA Block Weight":{
          "requirements_path": f"{extension_dir}/sd-webui-lora-block-weight/requirements.txt"
    },
    "Mov2mov":{
          "requirements_path": f"{extension_dir}/sd-webui-mov2mov/requirements.txt"
    },
    "ReActor for Stable Diffusion":{
          "requirements_path": f"{extension_dir}/sd-webui-reactor/requirements.txt"
    },
    "stable-diffusion-webui-rembg":{
        "requirements_path": f"{extension_dir}/stable-diffusion-webui-rembg/requirements.txt"
    },
     "sd-civitai-browser (CivBrowser)":{
         "requirements_path":  f"{extension_dir}/sd-webui-civbrowser/requirements.txt"
    },
     "sd_katanuki":{
          "requirements_path": f"{extension_dir}/sd_katanuki/requirements.txt"
    },
    "sd-webui-depth-lib":{
        "requirements_path": f"{extension_dir}/sd-webui-depth-lib/requirements.txt"
    },
    "sd-webui-enable-checker":{
        "requirements_path": f"{extension_dir}/sd-webui-enable-checker/requirements.txt"
    },
     "sd-webui-freeu":{
          "requirements_path": f"{extension_dir}/sd-webui-freeu/requirements.txt"
     },
     "SearchMyPNG":{
           "requirements_path": f"{extension_dir}/SearchMyPNG/requirements.txt"
     },
    "sd-webui-segment-anything":{
        "requirements_path": f"{extension_dir}/sd-webui-segment-anything/requirements.txt"
    },
     "Stable Diffusion webui Infinite Image Browsing":{
           "requirements_path": f"{extension_dir}/sd-webui-infinite-image-browsing/requirements.txt"
     },
    "SuperMerger":{
          "requirements_path": f"{extension_dir}/sd-webui-supermerger/requirements.txt"
    },
     "TrainTrain":{
          "requirements_path": f"{extension_dir}/sd-webui-traintrain/requirements.txt"
     },
    "Manga Editor Desu! Pro Edition":{
            "requirements_path": f"{extension_dir}/stable-diffusion-webui-simple-manga-maker/requirements.txt"
    }
}

# 拡張機能のrequirements.txtをインストール
print("Extensionのrequirements.txtをインストール開始")

for key, file_info in extension_choices.items():
    if os.path.exists(file_info.get("requirements_path")):
        print(f"{key}の拡張機能のrequirements.txtをインストールしています...")
        try:
          !pip install -r {file_info.get("requirements_path")}
        except Exception as e:
          print(f"Error installing requirements for {key}: {e}")
          continue # エラーが発生しても次の拡張機能に進む

print("すべての拡張機能のrequirements.txtのインストール完了")

# `basicsr` のインストール
!pip install basicsr

end_time = time.time()
elapsed_time = end_time - start_time
print(f"すべての処理が完全に完了しました！({elapsed_time:.2f}秒)")

## 9.Launch the Stable Diffusion WebUI

In [ ]:
#@title ### ★ 以下の "1. !python launch.py --share --xformers --enable-insecure-extension-access" を使用

# Web UIを実行
!python /content/stable-diffusion-webui/launch.py --share --xformers --enable-insecure-extension-access

### **1. `!python launch.py --share --xformers --enable-insecure-extension-access`**
- **注釈：**  
  - WebUIを起動し、以下のオプションを有効化します：
    1. `--share`：WebUIをインターネット経由で共有可能にする。
    2. `--xformers`：Xformersライブラリを利用して計算効率を向上。
    3. `--enable-insecure-extension-access`：セキュリティ制限を緩和して、全ての拡張機能にアクセスを許可。

- **最適な使用時：**  
  - **外部リンクで共有する必要があり、カスタム拡張機能を使用する場合。**
  - 信頼できる環境で、拡張機能をロードする際の制約を緩和する必要があるときに使用。  
    ※ セキュリティリスクが増すため、信頼できる拡張機能のみを使う場合に限ります。

In [ ]:
# # WebUIを外部共有し、Xformersを有効化。セキュリティ制限を緩和して拡張機能にアクセス可能にする
# !python /content/stable-diffusion-webui/launch.py --share --xformers --enable-insecure-extension-access

### **2. `!python launch.py --share --xformers`**
- **注釈：**  
  - WebUIを起動し、以下のオプションを有効化します：
    1. `--share`：WebUIをインターネット経由で共有可能にする。
    2. `--xformers`：Xformersライブラリを利用して計算効率を向上。

- **最適な使用時：**  
  - **外部リンクでWebUIを共有したい場合**。  
    Xformersを有効化してメモリ効率や生成速度を改善する目的で使用します。
  - 特に拡張機能のセキュリティ制限を緩和する必要がない場合に安全に使用可能。

In [ ]:
# # WebUIを外部共有し、Xformersを有効化。拡張機能のセキュリティ制限を緩和しない安全な方法
# !python launch.py --share --xformers

### **3. `!python /content/stable-diffusion-webui/launch.py --share --xformers`**
- **注釈：**  
  - `/content/stable-diffusion-webui/launch.py` をフルパスで指定して起動し、以下のオプションを有効化します：
    1. `--share`：WebUIをインターネット経由で共有可能にする。
    2. `--xformers`：Xformersライブラリを利用して計算効率を向上。

- **最適な使用時：**  
  - **起動ディレクトリが不明確な場合や、明示的にスクリプトのフルパスを指定したい場合。**
  - Colab環境でディレクトリを移動している可能性がある場合に使用すると安心。

In [ ]:
# # 明示的にフルパスを指定してWebUIを起動。外部共有とXformersを有効化
# !python /content/stable-diffusion-webui/launch.py --share --xformers

### **4. `!python /content/stable-diffusion-webui/launch.py --share`**
- **注釈：**  
  - `/content/stable-diffusion-webui/launch.py` をフルパスで指定して起動し、以下のオプションを有効化します：
    1. `--share`：WebUIをインターネット経由で共有可能にする。
  - **Xformersは有効化されていません**。

- **最適な使用時：**  
  - **Xformersを必要とせず、単純にWebUIを共有したい場合。**
  - メモリ効率の最適化が不要な場合や、Xformersがインストールされていない環境での利用に適しています。

In [ ]:
# # 明示的にフルパスを指定してWebUIを起動。Xformersを使用せず、外部共有のみを行う
# !python /content/stable-diffusion-webui/launch.py --share

### **その他. `!rm -rf /content/stable-diffusion-webui/tmp`**
- **注釈：**  
  - ディレクトリを完全に削除するコマンド。
  - キャッシュや一時ファイル（tempファイル）をクリアするために使用します。
  - 特定の不具合や古いキャッシュが原因でエラーが発生する場合に有効です。

- **最適な使用時：**  
  - Stable Diffusion WebUIを再起動する際に、不要な一時ファイルを削除してクリーンな状態にしたい場合。
  - モデルの切り替えや拡張機能を再インストールする際に、一時的なデータをクリアするために使用。

In [ ]:
# # キャッシュや一時ファイルを削除してクリーンな状態にする
# !rm -rf /content/stable-diffusion-webui/tmp

### **まとめ：コマンドの比較**
| 起動コマンド | 速度への影響 | 最適な使用状況 |
|----------|--------------|--------------|
| `!python launch.py --share --xformers --enable-insecure-extension-access` | **速い**（拡張機能次第でわずかに遅くなる場合あり）。 | 拡張機能を利用しつつ外部リンク共有する場合。 |
| `!python launch.py --share --xformers` | **最速**（拡張機能なし、効率化のみ）。 | 外部リンク共有が必要でXformersを有効化したい場合。 |
| `!python /content/stable-diffusion-webui/launch.py --share --xformers` | **最速**（パス指定が違うだけで同じ挙動）。 | 安全に外部共有かつフルパス指定で確実に起動したい場合。 |
| `!python /content/stable-diffusion-webui/launch.py --share` | **遅い**（`--xformers`がないため、計算効率が低下）。 | Xformersが不要な簡易な外部共有の場合。 |

---

| その他コマンド | 最適な使用状況 |
|----------|--------------|
| `!rm -rf /content/stable-diffusion-webui/tmp` | キャッシュクリアや不具合解消時に使用。 |

---
最も一般的で効率的なのは、**`!python launch.py --share --xformers`** です。必要に応じて拡張機能を使う場合は、`--enable-insecure-extension-access` を追加してください。

### **おすすめ**
- 拡張機能を使わない場合、**`!python launch.py --share --xformers`** を選択してください。
- 拡張機能を使用する場合、**`!python launch.py --share --xformers --enable-insecure-extension-access`** が最適です（速度を若干犠牲にして機能を拡張）。

## Reference
[safa-dayo/sd-webui-google-colab-setup.sh](https://gist.github.com/safa-dayo/b25f3737675c7667fefb51498dbf7212)